<a href="https://colab.research.google.com/github/towardsai/ai-tutor-rag-system/blob/main/notebooks/Agents_with_OpenAI_Assistants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q openai==1.59.6

In [ ]:
import os

# Set the "OPENAI_API_KEY" in the Python environment. Will be used by OpenAI client later.
os.environ["OPENAI_API_KEY"] = "[OPENAI_API_KEY]"


# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')

# Math Tutor


In [ ]:
from openai import OpenAI

client = OpenAI()

assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    model="gpt-4o",
    tools=[{"type": "code_interpreter"}],
)

In [ ]:
thread = client.beta.threads.create()

In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?",
)

In [ ]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

In [ ]:
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

In [ ]:
print(messages[0].content[0].text.value)

The solution to the equation \(3x + 11 = 14\) is \(x = 1\).


# Customer Support


In [ ]:
!wget https://personales.unican.es/corcuerp/linux/resources/LinuxCommandLineCheatSheet_1.pdf

--2025-01-22 10:40:57--  https://personales.unican.es/corcuerp/linux/resources/LinuxCommandLineCheatSheet_1.pdf
Resolving personales.unican.es (personales.unican.es)... 193.144.193.111
Connecting to personales.unican.es (personales.unican.es)|193.144.193.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 267816 (262K) [application/pdf]
Saving to: ‘LinuxCommandLineCheatSheet_1.pdf’

LinuxCommandLineChe 100%[===================>] 261.54K   407KB/s    in 0.6s    

2025-01-22 10:40:58 (407 KB/s) - ‘LinuxCommandLineCheatSheet_1.pdf’ saved [267816/267816]



In [ ]:
from openai import OpenAI

client = OpenAI()

In [ ]:
# Create a vector store caled "Financial Statements"
vector_store = client.beta.vector_stores.create(name="Tech Support")

# Ready the files for upload to OpenAI
file_streams = [open("LinuxCommandLineCheatSheet_1.pdf", "rb")]

# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
)

# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [ ]:
assistant = client.beta.assistants.create(
    name="Tech Support",
    instructions="You are a tech support chatbot. Use the product manual to respond accurately to customer inquiries.",
    model="gpt-4o",
    tools=[{"type": "file_search"}],
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [ ]:
# Create a thread and attach the file to the message
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "What 'ls' command do?",
        }
    ]
)

In [ ]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

In [ ]:
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

print(messages[0].content[0].text.value)

The `ls` command in Linux is used to list files and directories. Specifically, running `ls` without any options will list the files and directories in the current directory. Using `ls -al` provides a detailed listing of the contents, including hidden files【4:0†LinuxCommandLineCheatSheet_1.pdf】.


In [ ]:
messages[0].content[0].text.annotations

[FileCitationAnnotation(end_index=294, file_citation=FileCitation(file_id='file-GfFBhZk6qRgyUy1ixdiBJT'), start_index=256, text='【4:0†LinuxCommandLineCheatSheet_1.pdf】', type='file_citation')]